In [4]:
import random
import os
import glob

In [33]:
random_country_dir = random.choice(os.listdir("../country_data"))
random_country_dir

'SRB_Serbia'

In [35]:
figures = glob.glob(f"../country_data/{random_country_dir}/figures/*.png", recursive = True)
figures

['../country_data/SRB_Serbia/figures/SRB_BP_1.png',
 '../country_data/SRB_Serbia/figures/SRB_EIA_1.png',
 '../country_data/SRB_Serbia/figures/SRB_CAIT_gases_1.png',
 '../country_data/SRB_Serbia/figures/SRB_GCP_1.png',
 '../country_data/SRB_Serbia/figures/SRB_UNFCCC_NAI_1.png',
 '../country_data/SRB_Serbia/figures/SRB_CO2_totals.png',
 '../country_data/SRB_Serbia/figures/SRB_CDIAC_1.png',
 '../country_data/SRB_Serbia/figures/SRB_IEA_1.png']

In [36]:
random_fig = random.choice(figures)
random_fig

'../country_data/SRB_Serbia/figures/SRB_GCP_1.png'

In [37]:
print("random_fig path imported")

random_fig path imported
